In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sampling import *

import torch.utils.data as data
from torch.utils.data import DataLoader

import torch
import torch.optim as optim

import os
import torch
import pickle
from tqdm import tqdm

In [2]:
import argparse
from prettytable import PrettyTable

In [3]:
def print_args(parse_args):
    x = PrettyTable()
    x.field_names = ["Arg.", "Value"]
    for arg in vars(parse_args):
        x.add_row([arg, getattr(parse_args, arg)])
    print(x)

In [4]:
def arg_parse():
        parser = argparse.ArgumentParser(description='LiveRec - Douyu')

        parser.add_argument('--seed', dest='seed', type=int,
            help='Random seed')

        parser.add_argument('--batch_size', dest='batch_size', type=int,
            help='Batch size - only active if torch is used')
        parser.add_argument('--seq_len', dest='seq_len', type=int,
            help='Max size of the sequence to consider')

        parser.add_argument('--num_heads', dest='num_heads', type=int,
            help='Numer of heads to use for multi-heads attention')
        parser.add_argument('--num_heads_ctx', dest='num_heads_ctx', type=int,
            help='Numer of heads to use for multi-heads attention CTX')


        parser.add_argument('--dataset', dest='dataset', 
            help='Input dataset.')
 
        parser.add_argument('--model', dest='model', type=str,
            help='Type of the model')

        parser.add_argument('--model_from', dest='mfrom', type=str,
            help='Name of the model to load')
        parser.add_argument('--model_to', dest='mto', type=str,
            help='Name of the model to save')
        parser.add_argument('--cache_dir', dest='cache_dir', type=str,
            help='Path to save the cached preprocessd dataset')
 
        parser.add_argument('--model_path', dest='model_path', type=str,
            help='Path to save the model')
        parser.add_argument('--early_stop', dest='early_stop', type=int,
            help='Number of iteration without improvment before stop')
        parser.add_argument('--ev_sample', dest='ev_sample', type=int,
            help='Number of samples for the final evaluation')
        parser.add_argument('--device', dest='device', type=str,
            help='Pytorch device')

        parser.add_argument('--lr', dest='lr', type=float,
            help='Learning rate.')
        parser.add_argument('--mask_prob', dest='mask_prob', type=float,
            help='BERT mask prob.')
        parser.add_argument('--l2', dest='l2', type=float,
            help='Strength of L2 regularization')
        parser.add_argument('--dim', dest='K', type=int,
            help='Number of latent factors')
 
        parser.add_argument('--num_iters', dest='num_iter', type=int,
            help='Number of training iterations')
        parser.add_argument('--num_epochs', dest='num_epochs', type=int,
            help='Number of training epochs')
        parser.add_argument('--num_att', dest='num_att', type=int,
            help='Num attention module for seq encoding')
        parser.add_argument('--num_att_ctx', dest='num_att_ctx', type=int,
            help='Num attention for ctx module')
 
        parser.add_argument('--topk_att', dest='topk_att', type=int,
            help='Items to send to attentive output')

        parser.add_argument('--fr_ctx', dest='fr_ctx', nargs='?',
            const=True, default=True,
            help='')
        parser.add_argument('--r_rep', dest='fr_rep', nargs='?',
            const=True, default=False,
            help='')
        parser.add_argument('--uniform', dest='uniform', nargs='?',
            const=True, default=False,
            help='')
        parser.add_argument('--debug', dest='debug', nargs='?',
            const=True, default=False,
            help='')
#         parser.add_argument('--caching', dest='caching', nargs='?',
#             const=True, default=False,
#             help='')
        parser.add_argument('--caching', dest='caching', nargs='?',
            const=True, default=True,
            help='')

        parser.set_defaults(
                        seed=42,
                        dataset="",
                        lr=0.0005,
                        l2=0.1,  
                        mask_prob=0.5,  
                        batch_size=100, 
                        num_att=2,
                        num_att_ctx=2,
                        num_heads=4,
                        num_heads_ctx=4,
                        num_iter=200,
                        seq_len=16,  
                        topk_att=64,  
                        early_stop=15,  
                        K=64,  
                        num_epochs=150,
#                         model="LiveRec",
                        model="POP",
                        model_path="",
                        mto="liverec",
                        device="cuda",
                        cache_dir=""
                       )


        args, unknown = parser.parse_known_args()
        return args 

In [5]:
args = arg_parse()
args.device = torch.device(args.device)
args.device = 'cpu'

print_args(args)

+---------------+---------+
|      Arg.     |  Value  |
+---------------+---------+
|      seed     |    42   |
|   batch_size  |   100   |
|    seq_len    |    16   |
|   num_heads   |    4    |
| num_heads_ctx |    4    |
|    dataset    |         |
|     model     |   POP   |
|     mfrom     |   None  |
|      mto      | liverec |
|   cache_dir   |         |
|   model_path  |         |
|   early_stop  |    15   |
|   ev_sample   |   None  |
|     device    |   cpu   |
|       lr      |  0.0005 |
|   mask_prob   |   0.5   |
|       l2      |   0.1   |
|       K       |    64   |
|    num_iter   |   200   |
|   num_epochs  |   150   |
|    num_att    |    2    |
|  num_att_ctx  |    2    |
|    topk_att   |    64   |
|     fr_ctx    |   True  |
|     fr_rep    |  False  |
|    uniform    |  False  |
|     debug     |  False  |
|    caching    |   True  |
+---------------+---------+


In [6]:
INFILE = os.path.join(args.dataset,'douyu_10k_min_duration.csv')
#user,streamer_id,start,stop
cols = ["user","streamer","start","stop","duration"]
data_fu = pd.read_csv(INFILE, header=None, names=cols)
# data_fu = data_fu.head(10000)


# Add one for padding
data_fu.user = pd.factorize(data_fu.user)[0]+1
data_fu['streamer_raw'] = data_fu.streamer
data_fu.streamer = pd.factorize(data_fu.streamer)[0]+1
print("Num users: ", data_fu.user.nunique())
print("Num streamers: ", data_fu.streamer.nunique())
print("Num interactions: ", len(data_fu))

Num users:  11610
Num streamers:  47612
Num interactions:  5378125


In [7]:
data_fu.start.min(),data_fu.start.max(),data_fu.stop.min(),data_fu.stop.max()

(0, 87839, 1, 87840)

In [8]:
args.M = data_fu.user.max()+1 # users
args.N = data_fu.streamer.max()+2 # items
args.D = data_fu.duration.max()+1 # duration
args.T = data_fu.start.max()+1

data_temp = data_fu.drop_duplicates(subset=['streamer','streamer_raw'])
umap      = dict(zip(data_temp.streamer_raw.tolist(),data_temp.streamer.tolist()))

args.M ,args.N ,args.D,args.T

(11611, 47614, 2883, 87840)

In [9]:
# Splitting and caching
max_step = max(data_fu.start.max(),data_fu.stop.max())
print("Num timesteps: ", max_step)
args.max_step = max_step
args.pivot_1  = max_step-500
args.pivot_2  = max_step-250

Num timesteps:  87840


In [10]:
print("caching availability")
ts = {}
max_avail = 0
for s in range(max_step):
    all_av = data_fu[(data_fu.start<=s) & (data_fu.stop>s)].streamer.unique().tolist()
    ts[s] = all_av
    max_avail = max(max_avail,len(ts[s]))
args.max_avail = max_avail
args.ts = ts
print("max_avail: ", max_avail)

# ts.items()

caching availability
max_avail:  446


In [11]:
# Compute availability matrix of size (num_timesteps x max_available)
max_av   = max([len(v) for k,v in args.ts.items()])
max_step = max([k for k,v in args.ts.items()])+1
av_tens = torch.zeros(max_step,max_av).type(torch.long)
for k,v in args.ts.items():
    av_tens[k,:len(v)] = torch.LongTensor(v)
args.av_tens = av_tens.to(args.device)

args.av_tens.size()

torch.Size([87840, 446])

In [12]:
data_fu

,user,streamer,start,stop,duration,streamer_raw
0,1,1,81824,81825,2,1211089da67
1,1,2,73720,73721,1,eb5bf69ab60
2,1,1,81825,81834,18,1211089da67
3,1,2,73673,73676,4,eb5bf69ab60
4,1,1,81745,81750,9,1211089da67
...,...,...,...,...,...,...
5378120,11610,8849,21366,21368,2,80ec36cddc2
5378121,11610,8849,21364,21367,4,80ec36cddc2
5378122,11610,8849,21319,21332,24,80ec36cddc2
5378123,11610,8849,21362,21365,4,80ec36cddc2


In [13]:
def custom_collate(batch,args):
    # returns a [batch x seq x feats] tensor
    # feats: [padded_positions,positions,inputs_ts,items,users,targets,targets_ts]

    bs = len(batch)
    feat_len = len(batch[0])
    batch_seq = torch.zeros(bs,args.seq_len, feat_len, dtype=torch.long)
    for ib,b in enumerate(batch):
        for ifeat,feat in enumerate(b):
            batch_seq[ib,b[0],ifeat] = feat
    return batch_seq

In [14]:
class SequenceDataset(data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ]).to(device)
    ## padd
    batch = [ torch.Tensor(t).to(device) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0).to(device)
    return batch, lengths, mask

def get_sequences(_data, _p1, _p2, args, max_u=int(10e9)):
    data_list = []

    _data = _data[_data.stop<_p2].copy()
    
    grouped = _data.groupby('user')
    for user_id, group in tqdm(grouped):
        group = group.sort_values('start')
        group = group.tail(args.seq_len+1)
        if len(group)<2: continue

        group = group.reset_index(drop=True) 
        
        # Get last interaction
        last_el = group.tail(1)
        yt = last_el.start.values[0]
        group.drop(last_el.index,inplace=True)

        # avoid including train in test/validation
        if yt < _p1 or yt >= _p2: continue

        padlen = args.seq_len - len(group)

        # sequence input features
        positions  = torch.LongTensor(group.index.values)
        inputs_ts  = torch.LongTensor(group.start.values)
        items      = torch.LongTensor(group['streamer'].values)
        duration   = torch.LongTensor(group['duration'].values)
        users      = torch.LongTensor(group.user.values)
        bpad       = torch.LongTensor(group.index.values + padlen)
#         diff_du    = torch.LongTensor(np.diff(np.array(inputs_ts[1:].tolist() + [last_el.start.values[0]])))

        # sequence output features
        targets    = torch.LongTensor(items[1:].tolist() + [last_el.streamer.values[0]])
        targets_ts = torch.LongTensor(inputs_ts[1:].tolist() + [last_el.start.values[0]])

        data_list.append([bpad,positions,inputs_ts,items,users,targets,targets_ts,duration])
#         data_list.append([bpad,positions,inputs_ts,items,users,targets,targets_ts,duration,diff_du])
        # stop if user limit is reached
        if len(data_list)>max_u: break

    return SequenceDataset(data_list)

In [15]:
cache_tr = os.path.join(args.cache_dir,"douyu_10k_min_dur_tr.p")
cache_te = os.path.join(args.cache_dir,"douyu_10k_min_dur_te.p")
cache_va = os.path.join(args.cache_dir,"douyu_10k_min_dur_val.p")

mu = int(10e9)

In [18]:
datalist_tr = get_sequences(data_fu,0,args.pivot_1,args,mu)
datalist_va = get_sequences(data_fu,args.pivot_1,args.pivot_2,args,mu)
datalist_te = get_sequences(data_fu,args.pivot_2,max_step,args,mu)
    
pickle.dump(datalist_te, open(cache_te, "wb"))
pickle.dump(datalist_tr, open(cache_tr, "wb"))
pickle.dump(datalist_va, open(cache_va, "wb"))

100%|██████████████████████████████████████████████████████████████████████████| 11610/11610 [00:04<00:00, 2348.07it/s]


In [16]:
datalist_tr = pickle.load(open(cache_tr, "rb"))
datalist_va = pickle.load(open(cache_va, "rb"))
datalist_te = pickle.load(open(cache_te, "rb"))

In [17]:
train_loader = DataLoader(datalist_tr,batch_size=args.batch_size,
                              collate_fn=lambda x: custom_collate(x,args))
val_loader   = DataLoader(datalist_va,batch_size=args.batch_size,
                              collate_fn=lambda x: custom_collate(x,args))
test_loader  = DataLoader(datalist_te,batch_size=args.batch_size,
                              collate_fn=lambda x: custom_collate(x,args))

In [18]:
from eval import *
from data import *
from modify_duration_lstm_v2 import *
from modify_duration_lstm_v2 import get_model_type_new

In [19]:
# args.lr = 0.001
# args.l2 = 0.001

In [20]:
args.r_rep = args.fr_rep

In [21]:
args.model='LiveRec_dur_lstm_v2'
MPATH9,MODEL9 = get_model_type_new(args)
MPATH9,MODEL9
model9 = MODEL9(args).to(args.device)
optimizer = optim.Adam(model9.parameters(),lr=args.lr,weight_decay=args.l2)

best_val = 0.0
best_max = args.early_stop
best_cnt = best_max

In [22]:
print("training...")
for epoch in range(args.num_epochs):
    loss_all = 0.0; loss_cnt = 0
    model9.train()
    for data in tqdm(train_loader):
        data = data.to(args.device)
        optimizer.zero_grad()
    
        loss = model9.train_step(data)
            
        loss_all += loss.item()
        loss_cnt += (data[:,:,5]!=0).sum()
        
#         loss1 = loss.detach_().requires_grad_(True)
#         loss1.backward()
        loss.backward()
        optimizer.step()
            
        if torch.isnan(loss):
            print("loss is nan !") 
    
    scores = compute_recall(model9, val_loader, args, maxit=500)
    print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss_all/loss_cnt))
    print_scores(scores)
    
    hall = scores['all']['h01']
    if hall>best_val:
        best_val = hall
        torch.save(model9.state_dict(), MPATH9)
        best_cnt = best_max
    else:
        best_cnt -= 1
        if best_cnt == 0:
            break
    
model9= MODEL9(args).to(args.device)
model9.load_state_dict(torch.load(MPATH9))
    
scores = compute_recall(model9, test_loader, args)
print("Final score")
print("="*11)
print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss_all/loss_cnt))
print_scores(scores)
save_scores(scores,args)

training...


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [03:46<00:00,  1.99s/it]
17it [00:13,  1.23it/s]


Epoch: 000, Loss: 2.10021
all: h@1: 0.19829 h@5: 0.29896 h@10: 0.34228 ndcg@1: 0.19829 ndcg@5: 0.25074 ndcg@10: 0.26468
new: h@1: 0.00598 h@5: 0.04036 h@10: 0.07773 ndcg@1: 0.00598 ndcg@5: 0.02200 ndcg@10: 0.03389
rep: h@1: 0.33093 h@5: 0.47732 h@10: 0.52474 ndcg@1: 0.33093 ndcg@5: 0.40849 ndcg@10: 0.42386
ratio:  0.5918242830994509


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [03:47<00:00,  1.99s/it]
17it [00:13,  1.23it/s]


Epoch: 001, Loss: 1.59540
all: h@1: 0.19341 h@5: 0.30567 h@10: 0.36730 ndcg@1: 0.19341 ndcg@5: 0.25264 ndcg@10: 0.27238
new: h@1: 0.01046 h@5: 0.05531 h@10: 0.10463 ndcg@1: 0.01046 ndcg@5: 0.03521 ndcg@10: 0.05118
rep: h@1: 0.31959 h@5: 0.47835 h@10: 0.54845 ndcg@1: 0.31959 ndcg@5: 0.40260 ndcg@10: 0.42493
ratio:  0.5918242830994509


...

Epoch: 084, Loss: 0.88167
all: h@1: 0.32032 h@5: 0.53264 h@10: 0.61745 ndcg@1: 0.32032 ndcg@5: 0.43314 ndcg@10: 0.46083
new: h@1: 0.02541 h@5: 0.20179 h@10: 0.31689 ndcg@1: 0.02541 ndcg@5: 0.11374 ndcg@10: 0.15145
rep: h@1: 0.52371 h@5: 0.76082 h@10: 0.82474 ndcg@1: 0.52371 ndcg@5: 0.65343 ndcg@10: 0.67421
ratio:  0.5918242830994509


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [02:57<00:00,  1.55s/it]
17it [00:10,  1.61it/s]


Epoch: 085, Loss: 0.87658
all: h@1: 0.31910 h@5: 0.52227 h@10: 0.61501 ndcg@1: 0.31910 ndcg@5: 0.42798 ndcg@10: 0.45809
new: h@1: 0.02093 h@5: 0.18984 h@10: 0.31540 ndcg@1: 0.02093 ndcg@5: 0.10709 ndcg@10: 0.14782
rep: h@1: 0.52474 h@5: 0.75155 h@10: 0.82165 ndcg@1: 0.52474 ndcg@5: 0.64929 ndcg@10: 0.67208
ratio:  0.5918242830994509


24it [00:15,  1.58it/s]

Final score
Epoch: 085, Loss: 0.87658
all: h@1: 0.32123 h@5: 0.51752 h@10: 0.59941 ndcg@1: 0.32123 ndcg@5: 0.42629 ndcg@10: 0.45276
new: h@1: 0.01469 h@5: 0.15422 h@10: 0.24725 ndcg@1: 0.01469 ndcg@5: 0.08583 ndcg@10: 0.11572
rep: h@1: 0.48260 h@5: 0.70876 h@10: 0.78479 ndcg@1: 0.48260 ndcg@5: 0.60552 ndcg@10: 0.63019
ratio:  0.6551287463064585


In [16]:
# 定义模型
class TimeAwareAttentionMemory(torch.nn.Module):
    def __init__(self, embedding_dim, memory_dim, num_intervals, interval_dim, output_dim):
        super(TimeAwareAttentionMemory, self).__init__()
        self.num_intervals = num_intervals

        # 定义嵌入层
        self.item_embedding = nn.Embedding(num_items, embedding_dim)  # item嵌入
        self.interval_embedding = nn.Embedding(num_intervals, interval_dim)  # 间隔嵌入
        self.time_embedding = nn.Embedding(num_intervals, embedding_dim)  # 时间间隔嵌入

        # 定义记忆网络
        self.memory_network = nn.LSTM(embedding_dim, memory_dim, batch_first=True)  # LSTM记忆网络

        # 定义注意力网络和输出网络
        self.attention_network = nn.Sequential(
            nn.Linear(memory_dim + interval_dim, 1),  # 注意力网络
            nn.Softmax(dim=1)
        )
        self.output_network = nn.Linear(memory_dim, output_dim)  # 输出网络

    def forward(self, input_seq, interval_seq):
        # 嵌入输入序列
        input_emb = self.item_embedding(input_seq)  # 物品嵌入

        # 嵌入时间间隔序列
        interval_emb = self.interval_embedding(interval_seq)  # 间隔嵌入
        time_emb = self.time_embedding(interval_seq)  # 时间间隔嵌入

        # 嵌入时间间隔
        interval_len = input_seq.size(1) // self.num_intervals  # 每个间隔的长度
        intervals = (torch.arange(self.num_intervals) * interval_len).unsqueeze(0) + torch.arange(
            interval_len).unsqueeze(1)  # 间隔序列
        interval_emb = interval_emb[:, intervals]  # 每个间隔的嵌入
        time_emb = time_emb[:, intervals]  # 每个时间间隔的嵌入

        # 组合输入序列
        packed_input = nn.utils.rnn.pack_padded_sequence(input_emb, input_lens, batch_first=True, enforce_sorted=False)

        # 通过记忆网络处理输入序列
        memory, _ = self.memory_network(packed_input)  # LSTM的输出

        # 填充记忆向量序列
        memory, _ = nn.utils.rnn.pad_packed_sequence(memory, batch_first=True)  # LSTM的输出序列

        # 计算注意力分数
        time_emb = time_emb.unsqueeze(2).repeat(1, 1, memory.size(2))  # 将时间间隔的嵌入复制到与记忆向量匹配的维度
        attn_input = torch.cat([memory, interval_emb, time_emb], dim=2)  # 注意力计算的输入
        attn_scores = self.attention_network(attn_input)  # 计算注意力权重

        # 应用注意力到记忆向量
        attn_scores = attn_scores.unsqueeze(2)
        memory_attn = (attn_scores * memory).sum(dim=1)  # 注意力记忆向量

        # 通过输出网络处理记忆向量
        output = self.output_network(memory_attn)  # 最终输出
        return output